In [12]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser
from langchain_community.chat_models import ChatOllama


#### ROUTER
# This is the router - responsible for chosing what to do
chain = PromptTemplate.from_template("""Given the user question below, classify it as either being about `general_conversation` or `mediwave`
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:""") | ChatOllama(model='mistral:7b-instruct-v0.2-q6_K') | StrOutputParser()



In [15]:
chain.invoke({'question': 'current trends in ai'})

' general_conversation.'

In [3]:

#### Agent
# Defint the agent, which one branch of the router will use
from langchain.agents import XMLAgent, tool, AgentExecutor

model = ChatOllama(model="mistral:7b-instruct-v0.2-q6_K")

@tool
def search(query: str) -> str:
    """Search things about current events."""
    return "32 degrees"

tool_list = [search]


# Get prompt to use
prompt = XMLAgent.get_default_prompt()

# Logic for going from intermediate steps to a string to pass into model
# This is pretty tied to the prompt
def convert_intermediate_steps(intermediate_steps):
    log = ""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool><tool_input>{action.tool_input}"
            f"</tool_input><observation>{observation}</observation>"
        )
    return log


# Logic for converting tools to string to go in prompt
def convert_tools(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])


In [5]:

agent = (
    {
        "question": lambda x: x["question"],
        "intermediate_steps": lambda x: convert_intermediate_steps(x["intermediate_steps"])
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | model.bind(stop=["</tool_input>", "</final_answer>"])
    | XMLAgent.get_default_output_parser()
)


In [6]:

agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)


In [7]:

#### General chain
# Define a general chain, which will be used in other cases
general_chain = PromptTemplate.from_template("""Respond to the following question:

Question: {question}
Answer:""") | ChatOllama(model='mistral:7b-instruct-v0.2-q6_K')


In [8]:

#### Router
# Define the routing logic
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
  (lambda x: "weather" in x["topic"].lower(), agent_executor),
  general_chain
)


In [9]:

#### All together!
# Put it all together now
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

full_chain.invoke({"question":"whats the weather in SF"})

AIMessage(content=" I cannot provide you with an exact answer as I don't have real-time access to current weather data. However, San Francisco generally experiences a Mediterranean climate with cool, wet winters and warm, dry summers. For the most accurate and up-to-date weather information in San Francisco, I would recommend checking a reliable weather forecasting website or using a local weather app.")

# crew ai

In [11]:
!pip install crewai
!pip install 'crewai[tools]'

  Using cached crewai-0.22.5-py3-none-any.whl.metadata (13 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_openai-0.0.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached docstring_parser-0.15-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.23.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached opentelemetry_proto-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_api-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_semantic_conventions-0.44b0-py3-none-any.whl.metadata (2.2 kB)
Using cached crewai-0.22.5-py3-none-any.whl (51 kB)
Using cached instructor-0.5.2-py3-none-any.whl (33 kB)
Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
Using cached opentelemetry_exporter_otlp_prot

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key

# os.environ["OPENAI_API_KEY"] = "Your Key"

from crewai import Agent
from crewai_tools import SerperDevTool
from langchain_community.chat_models import ChatOllama
search_tool = SerperDevTool()

model_llm = ChatOllama(model='mistral:7b-instruct-v0.2-q6_K')

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  llm=model_llm,
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  llm=model_llm,
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)


Tasks

In [3]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [4]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential  # Optional: Sequential task execution is default
)

In [16]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'current trends in AI'})
print(result)



> Entering new CrewAgentExecutor chain...
 I should start by conducting extensive research on current AI trends in healthcare to identify the next big trend. I will use the "Search the internet" tool to gather information on this topic.

Action: Search the internet
Action Input: {'search_query': 'AI trends in healthcare 2023'} 


Search results: Title: The Current State of AI in Healthcare and Where It's Going in 2023
Link: https://healthtechmagazine.net/article/2022/12/ai-healthcare-2023-ml-nlp-more-perfcon
Snippet: AI Healthcare Use Cases in 2023 and Beyond · Natural Language Processing and Conversational AI · Automated Scheduling · Integrating Omics, EHRs and ...
---
Title: How Artificial Intelligence Could Reshape Health Care
Link: https://www.morganstanley.com/ideas/ai-in-health-care-forecast-2023
Snippet: Meanwhile, the industry's average estimated budget allocation to these technologies is projected to grow from 5.7% in 2022 to 10.5% in 2024.
---
Title: Five trends converging 

router